
# Superstore Business Analytics — CEO/PM Q&A Notebook (All-in-One)

This notebook is designed to **teach by doing**: every section starts with a real **business question**, lays out a **statistical plan**, runs the **code**, and then **interprets the result in plain business language**.

**Topics covered**
- Why Stats Matter in Business
- Descriptive Statistics
- Probability & Distributions
- Sampling & Estimation
- Hypothesis Testing
- Correlation & Simple OLS
- **Time Series** (trend, seasonality, growth, simple forecast)
- Extras: ABC/Pareto for products, outliers & risk tails

> **How to use**: Set `CSV_PATH` below to your Superstore CSV and run section-by-section in class.


In [ ]:

# ==== Setup & Imports ====
from pathlib import Path
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose

pd.set_option('display.float_format', lambda x: f'{x:,.3f}')

# EDIT ME: point to your CSV
CSV_PATH = 'path/to/your/superstore.csv'  # <-- CHANGE THIS



## Data Loading & Feature Engineering

We clean columns, parse dates, coerce numerics, and add teaching-friendly features.


In [ ]:

EXPECTED_COLS = [
    "Row ID","Order ID","Order Date","Ship Date","Ship Mode","Customer ID",
    "Customer Name","Segment","Country","City","State","Postal Code","Region",
    "Product ID","Category","Sub-Category","Product Name","Sales","Quantity",
    "Discount","Profit"
]

def standardize_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = (
        df.columns.str.strip().str.lower()
        .str.replace(r'[^0-9a-zA-Z]+', '_', regex=True)
        .str.strip('_')
    )
    return df

def parse_dates(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    for col in ["order_date","ship_date"]:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors="coerce", infer_datetime_format=True)
    return df

def coerce_types(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    for col in ["sales","discount","profit"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")
    if "quantity" in df.columns:
        df["quantity"] = pd.to_numeric(df["quantity"], errors="coerce").astype("Int64")
    return df

def add_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    if "order_date" in df.columns and df["order_date"].notna().any():
        df["order_year"]  = df["order_date"].dt.year
        df["order_month"] = df["order_date"].dt.month
        df["order_ym"]    = df["order_date"].dt.to_period("M").dt.to_timestamp()
    if {"sales","profit"}.issubset(df.columns):
        with np.errstate(divide='ignore', invalid='ignore'):
            df["margin_rate"] = np.where(df["sales"]!=0, df["profit"]/df["sales"], np.nan)
    if "discount" in df.columns:
        df["discount_bin"] = pd.cut(df["discount"].fillna(0.0),
                                    bins=[-0.01,0.0,0.2,0.4,1.0],
                                    labels=["0","(0,0.2]","(0.2,0.4]","(0.4,1]"])
    return df

def load_superstore(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    keep = [c for c in EXPECTED_COLS if c in df.columns]
    df = df[keep]
    df = standardize_columns(df)
    df = parse_dates(df)
    df = coerce_types(df)
    df = add_features(df)
    return df

df = load_superstore(CSV_PATH)
df.head()



## 1) Descriptive Statistics

**Business Question (PM):** *“Where are we making and losing money — by segment, region, and category?”*  
**Statistical Plan:** Summarize numeric variables; group KPIs by business dimensions; find top/bottom products.


In [ ]:

def numeric_summary(df, cols=("sales","profit","discount","quantity","margin_rate")):
    present = [c for c in cols if c in df.columns]
    return df[present].describe(percentiles=[0.25,0.5,0.75]).T

def segment_kpis(df):
    keys = [c for c in ["segment","region","category"] if c in df.columns]
    num_cols = [c for c in ["sales","profit","quantity","discount","margin_rate"] if c in df.columns]
    out = df.groupby(keys)[num_cols].agg(["count","sum","mean","median"])
    out.columns = ["_".join(col).strip("_") for col in out.columns.to_flat_index()]
    return out.sort_values(out.columns[0], ascending=False)

def top_n_products(df, n=10, by="profit"):
    cols = [c for c in ["product_id","product_name","category","sub_category","sales","profit"] if c in df.columns]
    agg = df[cols].groupby(["product_id","product_name","category","sub_category"], dropna=False)[["sales","profit"]].sum()
    return agg.sort_values(by=by, ascending=False).head(n).reset_index()

summ = numeric_summary(df).round(3)
kpis = segment_kpis(df).round(3)
tops = top_n_products(df, n=10, by="profit").round(2)

display(summ)
display(kpis.head(10))
display(tops)



### Interpretation (tell the story)
- **Overall scale & spread:** Use medians/IQR to avoid outlier bias; note if `profit` has heavy tails.  
- **Segment/Region/Category:** Identify the **best** (highest median/sum profit) and **worst** performers.  
- **Top products:** Are profits concentrated in a few SKUs? That suggests **Pareto/ABC** focus.

> **Decision hint:** Double down on high-margin segments/categories; investigate chronic loss-makers for repricing or delisting.



## 2) Probability & Distributions

**Business Question (CEO):** *“What’s the chance we lose money at different discount levels?”*  
**Statistical Plan:** Estimate empirical `P(Profit > 0)` by `discount_bin`; optionally use normal-approx on sales for intuition.


In [ ]:

def empirical_probability_profit_positive_by_discount(df):
    if not {"discount_bin","profit"}.issubset(df.columns):
        return pd.DataFrame()
    tmp = df.assign(profit_pos=df["profit"] > 0)
    return tmp.groupby("discount_bin")["profit_pos"].mean().to_frame("p_profit>0")

def normal_approx_prob(series: pd.Series, threshold: float, side: str="above"):
    s = series.dropna()
    if len(s) < 2: return np.nan
    mu, sd = s.mean(), s.std(ddof=1)
    if sd <= 0: return float(threshold >= mu) if side=="above" else float(threshold <= mu)
    z = (threshold - mu) / sd
    if side == "above": return 1 - stats.norm.cdf(z)
    if side == "below": return stats.norm.cdf(z)
    return 2*(1 - stats.norm.cdf(abs(z)))

p_by_disc = empirical_probability_profit_positive_by_discount(df).round(3)
display(p_by_disc)

if "sales" in df.columns:
    med = df["sales"].dropna().median()
    approx = normal_approx_prob(df["sales"], med, side="above")
    print(f"Normal approx: P(Sales > median) ≈ {approx:.3f}")



### Interpretation
- As discount increases, if `P(Profit>0)` **falls**, deep discounts are **riskier**.  
- Use as a **risk dial**: which discount bands keep us safely profitable?  
- The normal-approx demo is just intuition; rely on empirical results for decisions.

> **Decision hint:** Set discount policies where `P(Profit>0)` stays acceptably high; require approvals for deeper cuts.



## 3) Sampling & Estimation

**Business Question (Analyst):** *“Can we estimate mean profit precisely with a small sample?”*  
**Statistical Plan:** Draw a simple random sample; compute a bootstrap **95% CI** for mean profit; calculate sample size for a desired MOE.


In [ ]:

from typing import Tuple

def take_sample(df, n=500, random_state=42):
    n = min(n, len(df))
    return df.sample(n=n, random_state=random_state)

def bootstrap_mean_ci(series: pd.Series, n_boot: int = 2000, alpha: float = 0.05, random_state: int = 42) -> Tuple[float,float,float]:
    rng = np.random.default_rng(random_state)
    s = series.dropna().values
    if len(s) == 0: return (np.nan, np.nan, np.nan)
    boot = []
    for _ in range(n_boot):
        sample = rng.choice(s, size=len(s), replace=True)
        boot.append(sample.mean())
    lo, hi = np.percentile(boot, [100*alpha/2, 100*(1-alpha/2)])
    return (float(np.mean(s)), float(lo), float(hi))

def sample_size_mean(sd: float, margin_error: float, z: float = 1.96) -> int:
    if margin_error <= 0 or sd <= 0: return np.nan
    return int(np.ceil((z*sd / margin_error)**2))

sample = take_sample(df, n=500)
mean_profit, lo, hi = bootstrap_mean_ci(sample["profit"])
sd_profit = df["profit"].std()
n_needed = sample_size_mean(sd=sd_profit, margin_error=5.0)

print(f"Bootstrap 95% CI for mean Profit (n={len(sample)}): mean={mean_profit:,.2f}, CI=({lo:,.2f}, {hi:,.2f})")
print(f"Required n for ±$5 MOE (~95%): {n_needed}")



### Interpretation
- The **CI** tells us the plausible range for mean profit; if it’s narrow enough, the estimate is **decision-grade**.  
- The **sample-size** figure helps plan data collection or experiment scale.

> **Decision hint:** If leadership needs ±$5 precision, ensure samples meet or exceed the computed `n`.



## 4) Hypothesis Testing

**Business Question (CEO):** *“Do deep discounts (>20%) **reduce** average profit?”*  
**Statistical Plan:** Two-sample **t-test** comparing mean profit between `discount <= 0.2` and `> 0.2`. Also test **segments** (two-proportion z), **regions** (ANOVA), and **Category×Region** (chi-square).


In [ ]:

def ttest_profit_by_discount(df, split=0.2):
    low  = df.loc[df["discount"] <= split, "profit"].dropna()
    high = df.loc[df["discount"] >  split, "profit"].dropna()
    res = stats.ttest_ind(low, high, equal_var=False)
    return {
        "n_low": int(low.shape[0]), "n_high": int(high.shape[0]),
        "mean_low": float(low.mean() if len(low)>0 else np.nan),
        "mean_high": float(high.mean() if len(high)>0 else np.nan),
        "t_stat": float(res.statistic), "p_value": float(res.pvalue)
    }

def prop_test_profit_positive_by_segment(df):
    tmp = df.assign(pos=df["profit"]>0)
    counts = tmp.groupby("segment")["pos"].agg(["sum","count"]).sort_values("count", ascending=False)
    if counts.shape[0] < 2: return None
    (x1,n1),(x2,n2) = counts.iloc[0].tolist(), counts.iloc[1].tolist()
    p1,p2 = x1/n1, x2/n2
    p_pool = (x1+x2)/(n1+n2)
    se = (p_pool*(1-p_pool)*(1/n1 + 1/n2))**0.5
    z  = (p1 - p2) / se if se>0 else np.nan
    p  = 2*(1 - stats.norm.cdf(abs(z))) if np.isfinite(z) else np.nan
    return {"segments": counts.index[:2].tolist(), "p1": float(p1), "p2": float(p2), "z": float(z), "p_value": float(p)}

def anova_profit_by_region(df):
    groups = [g["profit"].dropna().values for _, g in df.groupby("region")]
    if len(groups) < 2: return None
    f, p = stats.f_oneway(*groups)
    return {"k_groups": len(groups), "F": float(f), "p_value": float(p)}

def chi_square_category_region(df):
    tab = pd.crosstab(df["category"], df["region"])
    chi2 = stats.chi2_contingency(tab)
    return {"chi2": float(chi2[0]), "p_value": float(chi2[1]), "dof": int(chi2[2]), "expected_shape": tab.shape}

res_t = ttest_profit_by_discount(df)
res_p = prop_test_profit_positive_by_segment(df)
res_a = anova_profit_by_region(df)
res_c = chi_square_category_region(df)

display(res_t)
display(res_p)
display(res_a)
display(res_c)



### Interpretation
- **T-test (discounts):** If `mean_high << mean_low` and `p_value < 0.05`, deep discounts are **statistically associated** with lower profits.  
- **Proportions (segments):** Significant `p_value` implies **profitability rate** differs between top segments → tailor strategies.  
- **ANOVA (regions):** Significant result → at least one region’s mean profit differs → investigate drivers (shipping, pricing, mix).  
- **Chi-square (Category×Region):** Significant → sales mix varies by region → regional assortment or marketing.

> **Decision hint:** Tighten discount controls; run targeted experiments by segment/region to confirm causality before policy changes.



## 5) Correlation & Simple OLS

**Business Question (CFO):** *“How do discount, quantity, and sales **relate** to profit? Can we get a quick predictive read?”*  
**Statistical Plan:** Compute Pearson & Spearman correlations; fit a compact OLS model.


In [ ]:

def pearson_spearman(df, x="discount", y="profit"):
    s = df[[x,y]].dropna()
    if len(s) < 3: return None
    r_pear, p_pear = stats.pearsonr(s[x], s[y])
    r_spear, p_spear = stats.spearmanr(s[x], s[y])
    return {"pearson_r": float(r_pear), "pearson_p": float(p_pear),
            "spearman_rho": float(r_spear), "spearman_p": float(p_spear), "n": int(len(s))}

def simple_ols(df, y="profit", X=("discount","quantity","sales")):
    cols = [c for c in X if c in df.columns]
    dat = df[[y] + cols].dropna()
    if dat.shape[0] < len(cols) + 3: return None
    Xmat = sm.add_constant(dat[cols])
    model = sm.OLS(dat[y], Xmat).fit()
    return {"nobs": int(model.nobs), "r2": float(model.rsquared),
            "coef": {k: float(v) for k, v in model.params.to_dict().items()},
            "pvalues": {k: float(v) for k, v in model.pvalues.to_dict().items()}}

corrs = pearson_spearman(df, x="discount", y="profit")
ols   = simple_ols(df, y="profit", X=("discount","quantity","sales"))

display(corrs)
display(ols)



### Interpretation
- **Correlation:** Sign and magnitude tell us direction/strength (watch for nonlinearity & confounders).  
- **OLS:** Coefficients show marginal associations holding other included variables constant. Use for **directional insight**, not causation.

> **Decision hint:** If discount coefficient is strongly negative and significant, prioritize pricing experiments; if quantity/sales coefficients are positive, consider volume-driving tactics (with margin guardrails).



## 6) Time Series (Trend, Seasonality, Growth, Simple Forecast)

**Business Question (CEO):** *“How are sales and profit trending over time? What do we expect next month?”*  
**Statistical Plan:** Aggregate by month; compute moving averages and YoY growth; decompose seasonality; produce a simple naive/MA forecast.


In [ ]:

# Monthly aggregates
ts = df.dropna(subset=["order_ym"]).groupby("order_ym")[["sales","profit"]].sum().sort_index()

# 3-month moving averages
ts["sales_ma3"]  = ts["sales"].rolling(3, min_periods=1).mean()
ts["profit_ma3"] = ts["profit"].rolling(3, min_periods=1).mean()

# YoY growth if ≥ 12 months
if len(ts) >= 13:
    ts["sales_yoy"]  = ts["sales"].pct_change(12)
    ts["profit_yoy"] = ts["profit"].pct_change(12)

display(ts.tail(12))

# Seasonal decomposition (additive) if enough points
decomp_summary = {}
if len(ts) >= 24:
    decomp = seasonal_decompose(ts["sales"], model="additive", period=12, two_sided=False, extrapolate_trend='freq')
    decomp_summary = {
        "trend_last": float(decomp.trend.dropna().iloc[-1]) if decomp.trend is not None else np.nan,
        "seasonal_peek": float(decomp.seasonal.groupby(ts.index.month).mean().max()),
        "seasonal_trough": float(decomp.seasonal.groupby(ts.index.month).mean().min()),
    }
decomp_summary


In [ ]:

# Simple forecasts: last-value (naive) and MA(3) extrapolation
def naive_forecast(series: pd.Series):
    return float(series.dropna().iloc[-1]) if len(series.dropna()) else np.nan

def ma3_forecast(series: pd.Series):
    s = series.dropna()
    if len(s) == 0: return np.nan
    return float(s.rolling(3, min_periods=1).mean().iloc[-1])

next_sales_naive = naive_forecast(ts["sales"])
next_sales_ma3   = ma3_forecast(ts["sales"])
next_profit_naive = naive_forecast(ts["profit"])
next_profit_ma3   = ma3_forecast(ts["profit"])

print(f"Next-month Sales (naive): {next_sales_naive:,.0f} | MA(3): {next_sales_ma3:,.0f}")
print(f"Next-month Profit (naive): {next_profit_naive:,.0f} | MA(3): {next_profit_ma3:,.0f}")



### Interpretation
- **Trend:** Use MA(3) and decomposition trend to describe direction (up/down/flat).  
- **Seasonality:** Peaks/troughs reveal planning windows (inventory, promos).  
- **Growth:** YoY increasing? That supports expansion decisions.  
- **Forecast:** Naive/MA give a **baseline**; refine with ARIMA/Prophet if needed.

> **Decision hint:** Align promotions to seasonal peaks, ensure capacity before expected upswings, and set targets using conservative (naive) vs optimistic (MA) baselines.



## 7) Extras — ABC/Pareto, Outliers & Risk Tails

**Business Question (Merchandising):** *“Which SKUs drive most of our profit, and where are extreme losses coming from?”*  
**Statistical Plan:** Pareto rank by cumulative profit (ABC); inspect tail losses.


In [ ]:

# Pareto on product profit
prod = df.groupby(["product_id","product_name"], dropna=False)["profit"].sum().sort_values(ascending=False).reset_index()
prod["cum_profit"] = prod["profit"].cumsum()
total_profit = prod["profit"].sum()
prod["cum_share"] = prod["cum_profit"] / total_profit if total_profit != 0 else np.nan

def abc_class(share):
    if pd.isna(share): return np.nan
    if share <= 0.80: return "A"
    if share <= 0.95: return "B"
    return "C"

prod["ABC"] = prod["cum_share"].apply(abc_class)
display(prod.head(15))

# Loss tail: worst 1% of line items by profit
cut = np.nanpercentile(df["profit"], 1)
tail_losses = df.loc[df["profit"] <= cut, ["order_id","product_name","category","sub_category","discount","profit"]].sort_values("profit").head(20)
display(tail_losses)



### Interpretation
- **ABC:** A small fraction of SKUs likely drives most profit (A items). Protect availability and margins there.  
- **Loss tail:** Identify patterns (e.g., deep discounts, certain categories) and set **guardrails**.

> **Decision hint:** Prioritize A-items for inventory/assortment; review pricing/discounts for chronic loss-makers.



## Wrap-Up & Recommendations

- **Descriptives** surfaced where we win/lose.  
- **Probability** quantified risk by discount.  
- **Sampling/CI** gave precision planning.  
- **Hypothesis tests** backed policy choices with evidence.  
- **Correlation/OLS** offered directional drivers.  
- **Time series** informed planning and simple forecasting.  
- **Pareto/Outliers** focused attention where it matters most.

**Next steps for the team**
1. Define discount guardrails based on `P(Profit>0)` and t-test outcomes.  
2. Pilot **segment/region-targeted** strategies to validate causality.  
3. Use the MA/naive forecasts to set next-month targets; refine with ARIMA later.  
4. Maintain an **A/B test calendar** to turn insights into controlled experiments.
